In [ ]:
# import built in modules
import sys
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib
import zipfile as zf

!pip install bresenham
from bresenham import bresenham
import pandas as pd
import time 
# !pip install tqdm -U --user
!pip install tqdm
# import tqdm
from tqdm import tqdm_notebook as tqdm
from scipy.stats import norm as norm_rv
from scipy.stats import rv_discrete 
from matplotlib.animation import FuncAnimation

In [ ]:
''' NOTE: This code will be used when running code on AWS, to unzip file'''
# files = zf.ZipFile("ec833_project3_odom.zip", 'r')
# files.extractall('')
# files.close()

In [ ]:
''' NOTE: Must NOT separate custom module imports from raw data import '''

# import custom modules + data

import load_data, dataloader

custom_modules = ["load_data","dataloader"]

# DELETE: assures up-to-date local modules are always used
for module_par in custom_modules:
    del sys.modules[module_par]

# RELOAD: assures up-to-date local modules are always used
import load_data, dataloader


# load all data from target folder
data_folder = "data"

# produces list with objects of class "map_object" in it
map_list_train = list(dataloader.load_folder(data_folder))

for map_number in range(len(map_list_train)):
    
    print("Map ID",map_list_train[map_number].id)
    print("Units are Meters. Robot is assumed to start facing right at coordinate (0,0)")
    fig1 = plt.figure(figsize=(15,15))
    plt.plot(map_list_train[map_number].cummulative_displacement[:,0],\
             map_list_train[map_number].cummulative_displacement[:,1])
    plt.xlabel('Robot Horizontal Location (meters)', fontsize=18)
    plt.ylabel('Robot Vertical Location (meters)', fontsize=18)
    plt.xticks(fontsize= 16)
    plt.yticks(fontsize= 16) 

    fig1.suptitle('Odometry Navigation Map {0}'.format(map_list_train[map_number].id), fontsize=20)
    fig1.tight_layout()
    fig1.subplots_adjust(top=0.95)
    fig1.savefig('maps/odometry_map{0}.png'.format(map_list_train[map_number].id))
    
    plt.show()

In [ ]:
top = '\n##########################################################'
bot = "##########################################################\n"

In [ ]:
def get_effective_n(weights_current):
    return 1 / np.sum(weights_current**2)
    
     #how to test this:
    '''a = np.array([[4],[6]])
    print(a**2)'''
    

In [ ]:
def normalize_weights(weights_resampled_par):
    weights_resampled_par /= np.sum(weights_resampled_par)
    return weights_resampled_par

    #how to test this:
    ''' weights_test = np.ones(10)/10
        weights_test /= 2
        print((weights_test[0]))'''
    

In [ ]:
def resample_particles(weights_current,particles_par):
    
    # get the number of weights
    weight_cnt = weights_current.shape[0]
    
    weights_cum = np.asarray([np.sum(weights_current[0:i+1]) for i in range(weight_cnt)])
    
    # produce n uniform random samples
    random_density_samples = np.random.rand(weight_cnt)
    
    list_add = []
    for sample in random_density_samples:
        list_add.append(np.sum(weights_cum < sample))
       
    randomly_sampled_indexes = np.asarray(list_add)
    
    weights_resampled  = weights_current[randomly_sampled_indexes]
    weights_resampled = normalize_weights(weights_resampled)
    
    particle_resampled = particles_par[randomly_sampled_indexes]

    return particle_resampled,weights_resampled
    
    #how to test this:
    '''print(np.argsort(weights_current),"np.argsort(weights_current)")
    print(np.argsort(np.bincount(randomly_sampled_indexes)),"np.argsort(np.bincount(randomly_sampled_indexes))")'''

In [ ]:
def plot_particles(x0, y0, theta0, x_noise, y_noise, theta_noise):
    
    # create a single 10 x 10 figure
    fig_particles, plt_particles = plt.subplots(figsize=(10,10))
    
    # print 
    title_particles = str("pose_angle: "+str(pose_angle_old_SLAM*180/np.pi)+\
                          " | d_theta from ODOMETRY "+str(d_theta*180/np.pi)+"| dx from ODOMETRY "+str(dx)+\
                          " dy from ODOMETRY"+ str(dy))
    fig_particles.suptitle(title_particles, fontsize=16)

    
    plt_particles.set_xlabel('distance (PIXELS)')
    plt_particles.set_ylabel('distance (PIXELS)')

    plt_particles.quiver(x0,y0,np.cos(theta0),np.sin(theta0))
    plt_particles.quiver(x_noise,y_noise,np.cos(theta_noise),np.sin(theta_noise), color='m', alpha=1, headwidth=.3*conv_factor)
    
    plt_particles.ticklabel_format(useOffset=False)
    
    plt.show()

    #     fig_particles.show()
    print("TIME STEP FOR PLOT PARTICLE ABOVE:",time_step)
    

In [ ]:
def make_lidar_rays(lidar_global_hit_locs_par,particles_par):
    
    # extract the 0th column of all particles, which represents the angles
    angle_matrix = particles_par[:,0]
    
    # make an array where each column is the consine and sine of the angle associated with each particle
    transform_matrix = np.asarray([np.cos(angle_matrix),np.sin(angle_matrix)])
    
    '''lidar_global_hit_locs_par contains the rotation matrix corresponding to each ray (1080 of them) in its local
    coordinate frame (angles from -135 to 135). The dot product below computes the transformation to the local frame'''
    global_lidar_vecs_matrix = np.dot(lidar_global_hit_locs_par,transform_matrix)
    #print(global_lidar_vecs_matrix.shape) # (1810 x 2 by number of particles)
    return global_lidar_vecs_matrix

In [ ]:
def update_map(points_impacted_par,map_array_par):
    map_array_update = np.copy(map_array_par)
    if map_array_update[points_impacted_par[-1]]<100:
            map_array_update[points_impacted_par[-1]] += 10    #* (angle_factor)
    
    for i in points_impacted_par[0:-1]:
        if map_array_update[i] > -100:
            map_array_update[i] += -.5   
    return map_array_update

In [ ]:
def make_noise(dx,dy,d_theta,particle_cnt):

    # make numbers between -1 and 1
    noise_x = 2 * (np.random.rand(particle_cnt)-0.5)
    # scale ONLY RANDOM noise by conversion factor 
    random_noise_x = x_random_noise_factor * noise_x * conv_factor
    # recalculate percent noise based on GLOBAL factor
    noise_x = factor_x * noise_x 

    # make numbers between -1 and 1
    noise_y = 2 * (np.random.rand(particle_cnt)-0.5)
    # scale ONLY RANDOM noise by conversion factor 
    random_noise_y =   y_random_noise_factor* noise_y * conv_factor
    # recalculate percent noise based on GLOBAL factor
    noise_y = factor_y * noise_y

    # make numbers between -1 and 1
    noise_theta = 2 * (np.random.rand(particle_cnt)-0.5)
    # scale ONLY RANDOM noise by conversion factor 
    random_noise_theta = theta_random_noise_factor * noise_theta
    # recalculate percent noise based on GLOBAL factor
    noise_theta = factor_theta * noise_theta            

    noised_d_x     = dx * (1 + noise_x)          + random_noise_x 
    noised_d_y     = dy * (1 + noise_y)          + random_noise_y 
    noised_d_theta = d_theta * (1 + noise_theta) + random_noise_theta 

    min_dx = np.amin(noised_d_x)
    max_dx = np.amax(noised_d_x)

    min_dy = np.amin(noised_d_y)
    max_dy = np.amax(noised_d_y)

    min_d_theta = np.amin(noised_d_theta)
    max_d_theta = np.amax(noised_d_theta)

    return noised_d_x, noised_d_y, noised_d_theta,\
           min_dx, min_dy, min_d_theta,\
           max_dx, max_dy, max_d_theta

In [ ]:
a = np.array([4,5,5,3])
print(np.unique(a))

In [ ]:
np.arange(1080/2-10,1080/2+12,4)

In [ ]:
# number of particles for model
particle_cnt = 100

# threhold for robot-body lidar noise, in meters, compared with lidar readings
robot_self_dist = .33

# +/- angle (with respect to straight ahead direaction) that is not expected to result in robot-body lidar noise
angle_reducer = 90

# pixels per meter
conv_factor = 10

# wdith of map in meters
map_width = 100

# holds lists of arrays, which are used to produce plots
map_plot_list = []


# hold list of lidar parameters
map_lidar_list = []

# hold numpy arrays of all maps
all_map_list = []

# number of (particle / map) plots allowed
particle_plot_cnt_allowed = 300

# numeric parameter of width of current map in pixels, will be a function parameter at the end 
map_cm = map_width * conv_factor
map_cm = map_cm + 1

# if the map width is not divisible by 2, issue warning
if ((map_cm - 1)%2) > 0:
    print ("ISSUE WITH THE MAP DIMENSION!") 

# center in x and y
x_center = int((map_cm - 1)/2)
y_center = np.copy(x_center)

# start plotting information at this time step
start_time_master = 1700

# complete this number of steps before stopping plotting
time_steps_master = 5000

# number of time steps between plots
plot_interval = 3

# hyperparameter save file parameter 
saved = 0

'''number from 1 to 10, representing the size of each random sample for angles at each time step
in multiples of 108'''
downsample = 5

# number of angles sampled for every time step
random_sample_cnt = 108*downsample

# iterate through all the maps
for idx in range(len(map_list_train)):
    
    # capture start time
    start = time.time()
    
    plot_array_list = []
    # initialize particle weights as *** 1/number of particles
    weights = np.ones(particle_cnt)/particle_cnt 
    
    # initialize weights old with the same uniform values
    weights_old = np.copy(weights)
    
    # print map id from filename
    print("Map ID:",map_list_train[idx].id)
    
#     if int(map_list_train[idx].id) != 24 and int(map_list_train[idx].id) != 23:
#         continue
    
#      if map is not 23,skip (for testing only)
#     if int(map_list_train[idx].id) == 23:
#         continue

#     if int(map_list_train[idx].id) != 20:
#         continue

#     if map is not 21,skip (for testing only)
#     if int(map_list_train[idx].id) != 21:
#         continue

#     # if map is not 20,skip (for testing only)
#     if int(map_list_train[idx].id) == 20:
#         continue
        
        
    # list to hold IMU data lined up with encoder and lidar data
#     acceleration_list = []
    
    
    print("Execution Started")

    # map nummpy array (to be populated)
    map_array = np.zeros([map_cm,map_cm])    

    # times, angles and ray return distances from a map id 'idx' 
    lidar_data = map_list_train[idx].lidar

    # Number of time observations for this map
    time_ob_cnt   = len(lidar_data)

    # time index where we start aligning lidar and encoder using a subsampled version of the encoder time array
    range_start = 3

    time_match_list = []

    ''' meas_per_tstep: get number of lidar measurements for every time step.
         lidar_angle_vec: get lidar angle vector array (same angles spanned for 
         all measurements, hence 17 hardcoded)

    17 is safely HARDCODED here because, no matter how many time steps there are, 
    there will be many more than 17 (picked that number just 'because') 
    all the measurements will have the same number of angular lidar measurements (1801)
    but didn't want to hardcode 1801, just in case.'''
    
    # lidar angle vector is from -135 to 135 ********in RADIANS*******
    meas_per_tstep  ,   lidar_angle_vec =   lidar_data[17]['angle'].shape[0]  ,    lidar_data[17]['angle']
    
    # make list of transformations for local to global coordinate frames
    ray_transform_vec =np.asarray([ [[np.cos(lidar_angle_vec[i]), -np.sin(lidar_angle_vec[i])],                                     [np.sin(lidar_angle_vec[i]),  np.cos(lidar_angle_vec[i])]]                                      for i in range(meas_per_tstep)]) 
    # transform list to array
    ray_transform_vec = np.squeeze(ray_transform_vec)
    
    '''print(ray_transform_vec[540+180])
       TESTED EQUALS [[ 0.70710678 -0.70710678]
                      [ 0.70710678  0.70710678]]'''
    
    #compute lidar off-center distance from wheelbase
    wheelbase_len   = 0.33020
    
    # locate distance of lidar lever from center of robot
    lidar_lever_len = 0.29833 - (wheelbase_len/2)
    
    # convert to small unit
    lidar_lever_len = lidar_lever_len * conv_factor

    # initialize old float (prior step) position for encoder
    x0_encoder_float_old = np.copy(x_center)
    y0_encoder_float_old = np.copy(y_center)
    
    # initialize old pose angle for encoder
    pose_angle_old = 0
    pose_angle_old_SLAM = 0
 
    # initialize old float (prior step) position for lidar basepoint
    x0_lidar_float_old = x_center + lidar_lever_len*np.cos(pose_angle_old)
    y0_lidar_float_old = y_center + lidar_lever_len*np.sin(pose_angle_old)

    # initialize old int (prior step) position for lidar basepoint. this is fine since the lidar
    # will hit a point different from 0,0 as soon as it starts running
    x1_lidar_int_old = 0
    y1_lidar_int_old = 0

    # counter of particle plots, to limit the number of plots generated at runtime
    particle_plot_cnt = 0
    
    # an array holding the ENCODER time values for the current map as indexed by idx
    time_array_encoder = map_list_train[idx].processed_encoder[:,0]
    
    # the IMU time for the current map as indexed by idx
    imu_time = np.transpose(map_list_train[idx].imu)[:,6]
    
    # the IMU data for the current map as indexed by idx
    imu_data = np.transpose(map_list_train[idx].imu)[:,:6]
    
    # random numbers for particle coloring, each color is 3 channels
    colors_rays = tuple(np.random.rand(particle_cnt,3))
    color_center = tuple(np.random.rand(3))
    
    list_test=[]
    
    displacements = map_list_train[idx].cummulative_displacement
    
    for time_step in tqdm(range(time_ob_cnt)):
        
        ## if we are testing, use this to stop after a certain number of time steps
#         if time_step > time_steps_master + start_time_master:
#             break
        
        # boolean variable: allow print if not too many plots have been generated 
        allow_print = False #particle_plot_cnt < particle_plot_cnt_allowed and time_step >= start_time_master and time_step % plot_interval == 0
        
        # list of hits for a time step based on the x,y and angle prediction
        hit_list = []
        
        # if time is less than a small integer, find the smallest difference between the encoder time array 
        # and current lidar time, return the index   
        if time_step < range_start:
            lidar_time_closest_arg = (np.abs(time_array_encoder - map_list_train[idx].lidar[time_step]['t'])).argmin()
        
        # if time is larger, find the smallest difference between as smaller version (size is determined by range_start) 
        # of the encoder time array and the current lidar time, return the index
        else:
            lidar_time_closest_small = time_array_encoder[lidar_time_closest_arg:lidar_time_closest_arg+range_start]
            lidar_time_closest_arg = (np.abs(lidar_time_closest_small - map_list_train[idx].lidar[time_step]['t'])).argmin() + lidar_time_closest_arg

        # robot pose angle in RADIANS
        pose_angle = map_list_train[idx].cummulative_angle[lidar_time_closest_arg]
        
#     TESTED: JUST COMMENT ALL BELOW THESE LIST/TWO PLOT LINES BELOW
#         list_test.append(pose_angle)
#     plt.plot(np.asarray(list_test)*180/np.pi)
#     plt.plot(np.zeros(len(list_test)))
#     plt.show()

     # get x coordinate of robot, initially based on encoder
        x0_encoder_float = displacements[lidar_time_closest_arg,0]

        # convert to small unit (i.e.: centimeters), converted  
        x0_encoder_float = x0_encoder_float * conv_factor

        # account for scaling center shift, converted 
        x0_encoder_float = x0_encoder_float + x_center
                
        # transfer to centroid of lidar, account for lidar being off center with encoder, BOTH IN SMALL UNITS
        x0_lidar_float = x0_encoder_float + lidar_lever_len*np.cos(pose_angle) 

        # get y coordinate of robot, initially based on encoder
        y0_encoder_float = displacements[lidar_time_closest_arg,1]

        # convert to small unit (i.e.: centimeters)    
        y0_encoder_float = y0_encoder_float * conv_factor

        # account for scaling center shift
        y0_encoder_float = y0_encoder_float + y_center    
        
        # transfer to centroid of lidar, account for lidar being off center with encoder
        y0_lidar_float= y0_encoder_float + lidar_lever_len*np.sin(pose_angle)
        
        x0_lidar_int = int(np.around(x0_lidar_float))
        y0_lidar_int = int(np.around(y0_lidar_float))
        
        # measure distance travelled in time step
        distance_traveled = np.linalg.norm(np.array([[y0_lidar_float-y0_lidar_float_old],[x0_lidar_float-x0_lidar_float_old]]))

        '''if distance travelled is less than one micrometer, 
        AND change is heading angle is less than .01 degree(s) ---> cell updates based on previous reading'''
        if time_step != 0 and distance_traveled < conv_factor * .00001 and abs(pose_angle-pose_angle_old)<np.pi/(18000): 
            map_array = update_map(points_impacted,map_array)
            continue

        factor_x = .8/20
        factor_y = .1/10 #.35
        factor_theta = 3/20
        
        #aqui
        x_random_noise_factor = 0.03072 
        x_random_noise_factor = x_random_noise_factor*12
          
        y_random_noise_factor = 0.025
        y_random_noise_factor = y_random_noise_factor /.65
        
        theta_random_noise_factor = 0.012
        theta_random_noise_factor = theta_random_noise_factor*6

        d_theta = (pose_angle - pose_angle_old)                    
        dx      = (x0_encoder_float - x0_encoder_float_old)
        dy      = (y0_encoder_float - y0_encoder_float_old)
        
        noised_d_x, noised_d_y, noised_d_theta,        min_dx,min_dy,min_d_theta,      \
        max_dx,max_dy,max_d_theta = make_noise(dx,dy,d_theta,particle_cnt)
        
        if saved == 0:
            saved = 1
            hyperparameters = ["random_sample_cnt ", str(random_sample_cnt),"\n",\
                               "particle_cnt ",str(particle_cnt),"\n",\
                               "conv_factor ",str(conv_factor),"\n",\
                               "factor_x ",str(factor_x),"\n",\
                               "factor_y ",str(factor_y),"\n",\
                               "factor_theta ", str(factor_theta),"\n",\
                               "x_random_noise_factor ", str(x_random_noise_factor),"\n",\
                               "y_random_noise_factor ", str(y_random_noise_factor),"\n",\
                               "theta_random_noise_factor ", str(theta_random_noise_factor)]
            
            hyper_filename = "hyper_parameters/hyperparameters_for_map_"+str(map_list_train[idx].id)+"_time_"+str(start)+".txt"
            with open (hyper_filename, 'w') as file:
                for i in hyperparameters:
                    file.write(i)
                file.close()
            
#             hyperparameters = np.savetxt(hyper_filename,np.asarray(hyperparameters))
            
        if get_effective_n(weights) < particle_cnt * .5:
#         if get_effective_n(weights) < 15:     
#             print("resampling particles. Effective n:", get_effective_n(weights),"Time Step",time_step,"\n",  )
#             print(np.argsort(weights))
            particles,weights_old = resample_particles(weights,particles)
        
        if time_step == 0:
            pt_no_noise = pose_angle # store old ANGLE
            px_no_noise = x0_encoder_float # store old x encoder origin
            py_no_noise = y0_encoder_float # store old y encoder origin
            
            theta_particles = pose_angle       + noised_d_theta # update encoder angle with noise
            x_particles     = x0_encoder_float + noised_d_x     # udpate encoder x with noise
            y_particles     = y0_encoder_float + noised_d_y     # update encoder y with noise 
 
        if time_step >0:  
            # parameters in particles before any new displacements or angle changes are added
            pt_no_noise = particles[:,0] # store particles before noise, ANGLE
            px_no_noise = particles[:,1] # store old x encoder origin for particles
            py_no_noise = particles[:,2] # store old y encoder origin for particles
            
            # parameters in particles after new displacements or angle changes are added
            theta_particles = particles[:,0] + noised_d_theta # update encoder angle with noise
            x_particles     = particles[:,1] + noised_d_x     # update encoder x with noise
            y_particles     = particles[:,2] + noised_d_y     # update encoder y with noise
            
        # score the particle data in "particles variable"
        particles = np.hstack((theta_particles[:,np.newaxis],x_particles[:,np.newaxis],y_particles[:,np.newaxis]))

        if False and allow_print: # if printing is allowed, pring the particles.
            particle_plot_cnt +=1
            plot_particles(px_no_noise,  py_no_noise  , pt_no_noise,\
                           particles[:,1], particles[:,2], particles[:,0])

        '''update "old_encoder", "old lidar" and "old_pose angle". ''' 
        x0_encoder_float_old = np.copy(x0_encoder_float)
        y0_encoder_float_old = np.copy(y0_encoder_float)
        
        x0_lidar_float_old   = np.copy(x0_lidar_float)
        y0_lidar_float_old   = np.copy(y0_lidar_float)
        
        pose_angle_old       = np.copy(pose_angle)
        pose_angle_old_SLAM  = np.copy(pose_angle_old)
        
        # update the scan vector based on current time step, IN SMALL UNITS!
        lidar_scan_vec  = lidar_data[time_step]['scan'] * conv_factor

        '''At current time step, multiplies the hits (distances in meters) for every angle  
        by the corresponding local (2x2) transform for that angle'''    
        # returns scaled cosine and sine components of reading along the robot's local coordinate frame, 
        # arranged in 2x2 matrices within a list, ready to transfer to global frame 
        
        '''lidar_scan_vec holds the readings from the lidar from angles -135 to 135
        ray_transform_vec holds the transformation matrix at each of those angles'''
        lidar_global_hit_locs = [lidar_scan_vec[i]*ray_transform_vec[i] for i in range(meas_per_tstep)]

        # converts this list reading to numpy array, in meters
        lidar_global_hit_locs = np.asarray(lidar_global_hit_locs)

        # adds offset to x lidar distance reading (based on the lidar off-center mounting distance), in meters
        lidar_global_hit_locs = lidar_global_hit_locs +  np.array([[lidar_lever_len,0],[0,lidar_lever_len]])
               
        # converts to array and squeeze extra dimension
        lidar_global_hit_locs = np.asarray(lidar_global_hit_locs)
        lidar_global_hit_locs = np.squeeze(lidar_global_hit_locs)
#         print(lidar_global_hit_locs.shape,"        print(lidar_global_hit_locs.shape)")
        
        '''A REDUCED MATRIX IS INTRODUCED HERE'''
        
        ray_indexer = np.random.choice(meas_per_tstep, random_sample_cnt)
        ray_indexer = np.hstack((ray_indexer,np.arange(1080/2-20,1080/2+20,1).astype(int)))
# #         print(ray_indexer)
        ray_indexer = np.unique(ray_indexer)
#         print(len(ray_indexer),"len")
        
        lidar_global_hit_locs_reduced = lidar_global_hit_locs[ray_indexer]
        
        if time_step > 1:

            # transfer all lidar rays for all particles to local frame
            lidar_ray_matrix_reduced = make_lidar_rays(lidar_global_hit_locs_reduced,particles)
            
            
            # for each time step, make the particle score zero to begin with
            particle_score = np.zeros(particle_cnt)
            
            
            for reading ,ray in enumerate(lidar_ray_matrix_reduced):
                ''' for this given time step and angle, distance is less than 0.33 meters and abs(angle) > 90 degrees, SKIP THIS READING'''
                if lidar_data[time_step]['scan'][reading] < robot_self_dist and (abs(180/np.pi*lidar_angle_vec[reading])> angle_reducer):
                    continue          
                
                #for each of the 1081 readings, set the x and y endpoints to ray values plus the center of each particle
                x1_rays_float = ray[0,:]  + particles[:,1]
                y1_rays_float = ray[1,:]  + particles[:,2]

                # make these vectors integers
                x1_rays_int = np.around(x1_rays_float).astype(int)
                y1_rays_int = np.around(y1_rays_float).astype(int)

#                 particle_score += np.multiply(map_array[y1_rays_int,x1_rays_int]>0,map_array[y1_rays_int,x1_rays_int])
                particle_score += map_array[y1_rays_int,x1_rays_int]

            shift = np.amax(particle_score)
            weights = np.exp(particle_score-shift)
            weights = weights/np.sum(weights)
            weights = np.multiply(weights,weights_old)
            weights = normalize_weights(weights)

            weights_old = np.copy(weights)
            
            index_max = np.argmax(particle_score)
            pose_angle       =  particles[index_max,0]
            x0_encoder_float =  particles[index_max,1]
            y0_encoder_float =  particles[index_max,2]

            if allow_print:
            
                print("\n####################################################")
                print("Odometry dx", dx,"for time", time_step)
                print("Minimum dx", min_dx,"for time", time_step)
                print("Maximum dx", max_dx,"for time", time_step)
                print("Choosen dx", noised_d_x[index_max],"for time", time_step,"\n")
                
                print("Odometry dy", dy,"for time", time_step)
                print("Minimum dy", min_dy,"for time", time_step)
                print("Maximum dy", max_dy,"for time", time_step)
                print("Choosen dy", noised_d_y[index_max],"for time", time_step,"\n")
                
                print("Odometry d_theta", d_theta,"for time", time_step)
                print("Minimum d_theta", min_d_theta,"for time", time_step)
                print("Maximum d_theta", max_d_theta,"for time", time_step)
                print("Choosen d_theta", noised_d_theta[index_max],"for time", time_step, "\n")
                
                print("####################################################\n")

            pose_angle_old_SLAM = np.copy(pose_angle)
 
            # transfer to centroid of lidar, account for lidar being off center with encoder, converted
            x0_lidar_float = x0_encoder_float + lidar_lever_len*np.cos(pose_angle) 

            # transfer to centroid of lidar, account for lidar being off center with encoder, converted
            y0_lidar_float = y0_encoder_float + lidar_lever_len*np.sin(pose_angle) 
            
            # convert all origin and end point to int variables
            x0_lidar_int = int(np.around(x0_lidar_float))
            y0_lidar_int = int(np.around(y0_lidar_float))
                    
        global_transform_vec = np.array([[np.cos(pose_angle)],[np.sin(pose_angle)]])
        
#          transform lidar to global frame for every reading, in meters


        lidar_global_hit_locs_reduced = [np.dot(lidar_global_hit_locs_reduced[i],global_transform_vec) for i in range(len(lidar_global_hit_locs_reduced))]

        # converts to array and squeeze extra dimension
        lidar_global_hit_locs_reduced = np.asarray(lidar_global_hit_locs_reduced)
#         print(lidar_global_hit_locs_reduced)
#         print(lidar_global_hit_locs_reduced.shape)
        lidar_global_hit_locs_reduced = np.squeeze(lidar_global_hit_locs_reduced)
        
        for reading in range(len(lidar_global_hit_locs_reduced)):
            if lidar_data[time_step]['scan'][reading] < robot_self_dist and\
            (abs(180/np.pi*lidar_angle_vec[reading])> angle_reducer):
                continue 
                    
            # shift by encoder location AND convert to smaller unit
            x1_lidar_float = np.squeeze(lidar_global_hit_locs_reduced[reading,0]) + x0_encoder_float 
            
#             print(x1_lidar_float,'x1_lidar_float')
            y1_lidar_float = np.squeeze(lidar_global_hit_locs_reduced[reading,1]) + y0_encoder_float


            # convert all end point to int variables
            x1_lidar_int = int(np.around(x1_lidar_float))
            y1_lidar_int = int(np.around(y1_lidar_float))

            ''' THIS IS DONE WITHIN SAME TIME STEP --> Origin has NOT CHANGED'''
            if x1_lidar_int == x1_lidar_int_old and y1_lidar_int == y1_lidar_int_old:
                continue

            x1_lidar_int_old = np.copy(x1_lidar_int)
            y1_lidar_int_old = np.copy(y1_lidar_int)
            hit_list.append([x1_lidar_int_old,y1_lidar_int_old])
            
            # generate list of impacted points
            '''Note that we use a (y0,x0,y1,x1 convention because of the way map_array is indexed (rows first))'''
            points_impacted = list(bresenham(y0_lidar_int,x0_lidar_int, y1_lidar_int,x1_lidar_int ))
            map_array = update_map(points_impacted,map_array)

        if allow_print: 
            particle_plot_cnt += 1
            
            plt.figure(figsize=(15,15))
            plt.imshow(map_array, origin="lower")
            plt.arrow(x0_encoder_float, y0_encoder_float,                      map_width*conv_factor/20*np.cos(pose_angle),                      map_width*conv_factor/20*np.sin(pose_angle),                      length_includes_head=True,    head_width=10, head_length=5)
            hit_vec_verify = np.asarray(hit_list)
            plt.scatter(hit_vec_verify[:,0],hit_vec_verify[:,1],c='r',alpha=1,s=1)
            plt.scatter(x0_lidar_int, y0_lidar_int,c='k',alpha=1)
            plt.show()
            print("TIME STEP FOR PLOT PARTICLE ABOVE:",time_step)
        
        if time_step % 40 == 0:
            plot_array_list.append([map_array,[x0_encoder_float,y0_encoder_float,pose_angle,time_step]])
            
    map_plot_list.append(plot_array_list)
    plt.close()

    all_map_list.append(np.copy(map_array))
    print("Total Execution Time: {0}".format(time.time()-start) )       
    
    
    fig = plt.figure(figsize=(15,15))
    map_copy = np.copy(map_array)

    extent=(-map_width/2,map_width/2,-map_width/2,map_width/2)
    plt.imshow(map_copy,origin = 'lower', extent = extent,cmap='binary')
    plt.xlabel('Robot Horizontal Location (meters)', fontsize=18)
    plt.ylabel('Robot Vertical Location (meters)', fontsize=18)
    plt.xticks(fontsize= 16)
    plt.yticks(fontsize= 16) 

    fig.suptitle('SLAM Occupancy Grid for Map {0}'.format(map_list_train[idx].id), fontsize=20)
    fig.tight_layout()
    fig.subplots_adjust(top=0.95)
    
    
    fig.savefig('maps/map{0}_time_{1}.png'.format(map_list_train[idx].id,start))

In [ ]:
top = 50
bottom = 40
left = 30
right = 50

ann_list = []

def update(i):
    
    for j, a in enumerate(ann_list):
        a.remove()
    ann_list[:] = []
    
    print(str(i/len(map_current)*100)[:5],"% Complete")

    im_normed = map_current[i][0]
    quiver_pars = map_current[i][1][:2]
    angle =  map_current[i][1][2]
    ax.imshow(im_normed, origin= "lower",extent=extent)
    t = map_current[i][1][3]
    
    
    if i == 0:
        rect = matplotlib.patches.Rectangle((right, bottom), left-right,top-bottom, angle=0.0,color=(1,1,1),)
        ax.add_patch(rect)
        
        texting = plt.annotate('T = '+str(int(t/40))+" sec",(.5*(left+right), .5*(top+bottom)),\
            ha='center',va='center', fontsize=14, color='red')
        
        ann_list.append(texting)
        
        
        ax.scatter((quiver_pars[0]-x_center)/conv_factor,(quiver_pars[1]-y_center)/conv_factor,c='r',s=5)
        return ax
    
    if i > 0:
        rect = matplotlib.patches.Rectangle((right, bottom), left-right,top-bottom, angle=0.0,color=(1,1,1),zorder = 3+i)
        ax.add_patch(rect)
        
        texting = plt.annotate('T = '+str(int(t/40))+" sec",(.5*(left+right), .5*(top+bottom)),\
            ha='center',va='center', fontsize=14, color='red',zorder = 4+i)
        
        ann_list.append(texting)

        
        ax.scatter((quiver_pars[0]-x_center)/conv_factor,(quiver_pars[1]-y_center)/conv_factor,c='r',s=5)
        return ax
    
    #     ax.set_title("Angle: {}*pi/10".format(i), fontsize=20)
#     ax.set_axis_off()

for index, map_current in enumerate(map_plot_list):
    extent=(-map_width/2,map_width/2,-map_width/2,map_width/2)
    fig, ax = plt.subplots(figsize=(15, 15))
    ax.set_title('Robot Location Versus Time for Map'+str(map_list_train[index].id))
    ax.set_xlabel('Robot Horizontal Location (meters)', fontsize=18)
    ax.set_ylabel('Robot Vertical Location (meters)', fontsize=18)
#     quiver_matrix = np.asarray(map_current)[:,1]
#     quiver_matrix = np.asarray([np.asarray(i) for i in quiver_matrix])
#     print(quiver_matrix.shape,"shape")
#     print(quiver_matrix)

#     ax.set_xticks(fontsize= 16)
#     ax.yticks(fontsize= 16) 
    print("Figure",map_list_train[index].id)
    anim = FuncAnimation(fig, update, frames=np.arange(0, len(map_current)), interval=300,repeat=True)
    anim.save('gifs/path_gif_map'+str(map_list_train[index].id)+".gif", dpi=80, writer='imagemagick')
    plt.close()
    print("DONE!")

In [ ]:
# !tar chvfz notebook.tar.gz 'maps'